In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
import os
os.getcwd()

'C:\\Users\\yashd\\OneDrive\\Desktop\\Online Internship\\ACMEgrade\\1. PROJECT\\1. Movie Reccomondation Engine'

# Load The data

In [3]:
path = r"C:\Users\yashd\OneDrive\Desktop\Online Internship\ACMEgrade\1. PROJECT\1. Movie Reccomondation Engine\dataset"

In [4]:
data = pd.read_csv(f"{path}\movies.csv")

In [5]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
display(data.shape)

(4803, 24)

In [7]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [8]:
# get the relevant features for recommendation
selected_features = ['id', 'title', 'genres', 'keywords', 'tagline', 'cast', 'director']

In [9]:
data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [10]:
# replace null values with empty string ('')

for feature in selected_features:
    data[feature] = data[feature].fillna('')

In [11]:
data[selected_features].isna().sum()

id          0
title       0
genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [12]:
new_data = data[selected_features]

In [13]:
display(new_data.head())

,id,title,genres,keywords,tagline,cast,director
0,19995,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [14]:
# combine all the features
combined_features = new_data["genres"] + " " + new_data["keywords"] + " " + new_data["tagline"] + " " + new_data["cast"] + " " + new_data["director"] 

In [15]:
combined_features.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
dtype: object

In [16]:
type(combined_features)

pandas.core.series.Series

In [ ]:
# (max_features=5000,stop_words='english')

In [19]:
# convert these features into vector form 
vectorizer = TfidfVectorizer(stop_words='english')
feature_vector = vectorizer.fit_transform(combined_features)
display(feature_vector.shape)

(4803, 17079)

In [20]:
# get the cosine similarity score by Cosine similarity
similarity = cosine_similarity(feature_vector)
print(similarity)

[[1.         0.04720973 0.03891121 ... 0.         0.         0.        ]
 [0.04720973 1.         0.021542   ... 0.01267122 0.         0.        ]
 [0.03891121 0.021542   1.         ... 0.         0.05626942 0.        ]
 ...
 [0.         0.01267122 0.         ... 1.         0.         0.02730605]
 [0.         0.         0.05626942 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02730605 0.         1.        ]]


In [21]:
print(similarity.shape)

(4803, 4803)


In [23]:
movie_name = input("Enter the movie name: ")

Enter the movie name:  man


In [24]:
# get all the movie name in a list
movie_list = new_data['title'].tolist()
movie_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [25]:
# get the similar names of the movie
find_colse_match = difflib.get_close_matches(movie_name, movie_list)
print(find_colse_match)

['Taxman', 'Pan', 'Hitman']


In [26]:
# get only first result
close_match = find_colse_match[0]
print(close_match)

Taxman


In [ ]:
# find the index of movie title
# for movie_name in movie_list:
#     if movie_name == close_match:
#         index = data[data['title'] == movie_name]['index'].values[0]
#         print(index)

In [36]:
data[data['title'] == close_match]['index']

3851    3851
Name: index, dtype: int64

In [37]:
index_of_movies = data[data['title'] == close_match]['index'].values[0]
print(index_of_movies)

3851


In [38]:
# get the similarity scores 
similarity_score = list(enumerate(similarity[index_of_movies]))
similarity_score

[(0, 0.005771455222128644),
 (1, 0.018625780768051787),
 (2, 0.014599559686777366),
 (3, 0.04193973659341018),
 (4, 0.006113080591821117),
 (5, 0.02209093230667159),
 (6, 0.0),
 (7, 0.019361406315354553),
 (8, 0.010340969968276064),
 (9, 0.005770411424137525),
 (10, 0.006469129898105092),
 (11, 0.019089817644812372),
 (12, 0.006040728787869534),
 (13, 0.005529963875034157),
 (14, 0.016308842116339935),
 (15, 0.0),
 (16, 0.018380342068960998),
 (17, 0.005681823476964591),
 (18, 0.019733528484397078),
 (19, 0.005025619253927614),
 (20, 0.005747182198109185),
 (21, 0.00514290386599047),
 (22, 0.0),
 (23, 0.0),
 (24, 0.005568775207613746),
 (25, 0.005853311758302622),
 (26, 0.019975358355856728),
 (27, 0.010299134230001273),
 (28, 0.01116811263323263),
 (29, 0.011782575209078525),
 (30, 0.022089809171468917),
 (31, 0.019251147303916038),
 (32, 0.0),
 (33, 0.01136786394017237),
 (34, 0.0),
 (35, 0.01662755006315759),
 (36, 0.015372477852376085),
 (37, 0.0),
 (38, 0.006051844096142293),
 (39

In [39]:
print(len(similarity_score))

4803


In [40]:
# movies based on the similarity score
sort_similarities_movies = sorted(similarity_score, key= lambda x : x[1], reverse=True)

In [41]:
# print the name of the movies based on the index top -30
i = 1
for movie in sort_similarities_movies:
    index = movie[0]
    movie_by_index = data[data.index == index]['title'].values[0]
    if i < 30:
        print(i,"", movie_by_index)
        i+=1

1  Taxman
2  28 Days
3  Hollywood Shuffle
4  Bad Boys
5  The Fugitive
6  Baby's Day Out
7  Showgirls
8  The Adventures of Pluto Nash
9  Once Upon a Time in America
10  Kill the Messenger
11  Bound
12  Parker
13  Public Enemies
14  Sherlock Holmes: A Game of Shadows
15  Eagle Eye
16  Fantastic 4: Rise of the Silver Surfer
17  No Reservations
18  Memento
19  Criminal Activities
20  Shade
21  The Pink Panther
22  Analyze This
23  When the Game Stands Tall
24  Happy Gilmore
25  Final Destination 3
26  Stuck on You
27  Seven Pounds
28  Conversations with Other Women
29  Limbo


In [42]:
def recommend(movie_name):
    
    find_colse_match = difflib.get_close_matches(movie_name, movie_list)[0]
    index_of_movies = data[data['title'] == find_colse_match]['index'].values[0]
    
    similarity_score = list(enumerate(similarity[index_of_movies]))
    sort_similarities_movies = sorted(similarity_score, key= lambda x : x[1], reverse=True)[:30]

    for i in sort_similarities_movies:
        print(data.iloc[i[0]].title)

In [43]:
movie = input("Enter the movie name: ")
recommend(movie)

Enter the movie name:  avatar


Avatar
Alien
Guardians of the Galaxy
Aliens
Star Trek Beyond
Star Trek Into Darkness
Galaxy Quest
Alien³
Gravity
Cargo
Trekkies
Moonraker
Jason X
Pocahontas
Space Cowboys
Lockout
Machete Kills
The Helix... Loaded
Event Horizon
Space Dogs
Gettysburg
Clash of the Titans
The Astronaut's Wife
Planet of the Apes
Star Wars: Clone Wars: Volume 1
The Right Stuff
Imaginary Heroes
Wing Commander
Star Trek
Terminator Salvation


In [46]:
name = input("Enter the movie name: ")

movie_list = new_data['title'].tolist()

find_colse_match = difflib.get_close_matches(name, movie_list)[0]

index_of_movies = data[data['title'] == find_colse_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movies]))

sort_similarities_movies = sorted(similarity_score, key= lambda x : x[1], reverse=True)

print("Suggested Movies for you: ")

i = 1
for movie in sort_similarities_movies:
    index = movie[0]
    movie_by_index = new_data[new_data.index == index]['title'].values[0]
    if i < 30:
        print(i,"", movie_by_index)
        i+=1

Enter the movie name:  iron man


Suggested Movies for you: 
1  Iron Man
2  Iron Man 2
3  Iron Man 3
4  Avengers: Age of Ultron
5  The Avengers
6  Captain America: Civil War
7  Captain America: The Winter Soldier
8  Ant-Man
9  X-Men
10  Made
11  X2
12  X-Men: Apocalypse
13  X-Men: First Class
14  The Incredible Hulk
15  The Helix... Loaded
16  Captain America: The First Avenger
17  Guardians of the Galaxy
18  Kick-Ass 2
19  Thor: The Dark World
20  Deadpool
21  X-Men: The Last Stand
22  X-Men: Days of Future Past
23  G-Force
24  Duets
25  Mortdecai
26  The Last Airbender
27  Southland Tales
28  Sky Captain and the World of Tomorrow
29  Zathura: A Space Adventure


In [47]:
path = r"C:\Users\yashd\OneDrive\Desktop\Online Internship\ACMEgrade\1. PROJECT\1. Movie Reccomondation Engine\models"

In [48]:
import pickle
pickle.dump(new_data, open(f"{path}\movies.pkl", 'wb'))

In [49]:
pickle.dump(similarity, open(f"{path}\similarity.pkl", 'wb'))

In [ ]:
# def get_top_30_movies(data, movie_name, selected_features, top_number=30):

#     # get the selected features and replace null by ''
#     for feature in selected_features:
#         data[feature] = data[feature].fillna('')

#     # combine all features
#     combined_features = " "
#     for feature in selected_features:
#         combined_features += pd.Series(data[feature]) + " "

#     # convert these features into vector form 
#     vectorizer = TfidfVectorizer()
#     feature_vector = vectorizer.fit_transform(combined_features)

#     # get the cosine similarity score by Cosine similarity
#     similarity = cosine_similarity(feature_vector)

#     # get all the movie name in a list
#     movie_list = data['title'].tolist()

#     # get the similar names of the movie
#     # get only first result
#     find_colse_match = difflib.get_close_matches(movie_name, movie_list)[0]

#     index_of_movies = data[data['title'] == find_colse_match]['index'].values[0]

#     # get the similarity scores 
#     similarity_score = list(enumerate(similarity[index_of_movies]))

#     # movies based on the similarity score
#     sort_similarities_movies = sorted(similarity_score, key= lambda x : x[1], reverse=True)

#     # print the name of the movies based on the index top -30
#     i = 1
#     for movie in sort_similarities_movies:
#         index = movie[0]
#         movie_by_index = data[data.index == index]['title'].values[0]
#         if i < top_number:
#             print(i,"", movie_by_index)
#             i+=1


# name = input("Enter the movie name: ")

# features = ['genres', 'keywords', 'tagline', 'cast', 'director']

# get_top_30_movies(data=data, movie_name=name ,selected_features=features, top_number=30)